In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
def crime_cleaner(data):
    # Merge key with crime db to gain state / city names
    washing = data.merge(key00,left_on=['FIPS_CTY','FIPS_ST'], right_on=['County FIPS Code','State FIPS Code'])
    # Rearrange columns
    washing = washing[['STUDYNO','EDITION','PART','IDNO','FIPS_ST','State FIPS Code','State Abbreviation','FIPS_CTY','County FIPS Code','GU Name','CPOPARST','AG_ARRST','JURFLAG','COVIND', 'GRNDTOT','P1TOT','P1VLNT','P1PRPTY','MURDER','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY']]
    
    # Select only wanted columns
    clean = washing[['State Abbreviation','GU Name','MURDER','P1TOT','P1VLNT','P1PRPTY','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY']]
    return clean
def wage_cleaner(data):
        # Drop Unwanted columns
    base = data.drop(['Area\nCode','Own','NAICS','Qtr','Status Code'],axis=1)
    # Select County level data
    washing = base.loc[(base['Area Type'] == 'County')]
    washing['Cnty'] = washing['Cnty'].astype('int64') # Convert to int for merging // Throws error
    washing['St'] = washing['St'].astype('int64')    # Convert to into for merging // Throws error
    # Merge with key
    washing = washing.merge(key00,left_on=['St','Cnty'], right_on=['State FIPS Code','County FIPS Code'])
    # Rearrange columns and drop unwanted columns
    county_wages = washing[['State Abbreviation','GU Name', 'Year', 'Area Type', 'St Name', 'Area', 'Ownership','Industry', 'Establishment Count', 'July Employment','August Employment', 'September Employment', 'Total Quarterly Wages','Average Weekly Wage', 'Employment Location Quotient Relative to U.S.','Total Wage Location Quotient Relative to U.S.']]
    county_wages['Establishment Count'] = county_wages['Establishment Count'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['Establishment Count'] = pd.to_numeric(county_wages['Establishment Count'])   # Convert to interger // Throws Error
    county_wages['September Employment'] = county_wages['September Employment'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['September Employment'] = pd.to_numeric(county_wages['September Employment'])    # Convert to interger // Throws Error  
    county_wages['Average Weekly Wage'] = county_wages['Average Weekly Wage'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['Average Weekly Wage'] = pd.to_numeric(county_wages['Average Weekly Wage'])  # Convert to interger // Throws Error
    county_wages['Industry'] = county_wages['Industry'].str.replace('0','') # Formatting to fix year 2018 industry codes
    county_wages['Industry'] = county_wages['Industry'].str.replace('1','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('2','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('3','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('4','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('5','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('6','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('7','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('8','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('9','')
    
#     state_wages = base.loc[(base['Area Type'] == 'State')]   # State Level Wages
#     nation_wages = base.loc[(base['Area Type'] == 'Nation')]    # Nation Level Data
#     urban_wages = base.loc[(base['Area Type'] == 'MSA')]   # Metropolitan Level Data
    return county_wages #,state_wages,nation_wages,urban_wages

def search(county,state):
    db10 = db2010.loc[(db2010['State Abbreviation'] == state) & (db2010['GU Name'] == county)]
    db12 = db2012.loc[(db2012['State Abbreviation'] == state) & (db2012['GU Name'] == county)]
    db14 = db2014.loc[(db2014['State Abbreviation'] == state) & (db2014['GU Name'] == county)]
    db16 = db2016.loc[(db2016['State Abbreviation'] == state) & (db2016['GU Name'] == county)]
    db18 = economic18.loc[(economic18['State Abbreviation'] == state) & (economic18['GU Name'] == county)]
    return db10,db12,db14,db16

In [ ]:
def searchcrime_cleaner(data,crimes,tsearch):
    container = []
    count = 0
    for i in range(len(data)):
        knight = data[count][tsearch].values
        container.append(knight)
        count += 1
    container = container[0][0],container[1][0],container[2][0],container[3][0]
    return container

In [3]:
# Function for cleaning data from search variable
def search_industrydata_cleaner(data,stat,tsearch):
    container = []
    count = 0
    for n in range(len(data)):    # Loop for pulling data of multiple years      
        container.append(data[n].loc[(data[n]['Industry']==tsearch)][stat].values[0])
#                          dictionary//year       dictionary//year//column    comparison/DATA TO PULL
        count += 1
    #container = container[0][0],container[1][0],container[2][0],container[3][0],container[4][0]
    return container # Return a list of individual values

# Function to find largest industries per county
def most_establishments(data):
    data = data.loc[(data['Industry'] != '10 Total, all industries')] # Remove Totals, 2018
    data = data.loc[(data['Industry'] != 'Total, all industries')] # Remove Totals, all other years
    data = data.nlargest(4,'Establishment Count')
    return data

In [1]:
# Function for cleaning data sample database
def industrydata_cleaner(data,stat,tsearch):
    container = []
    for i in data:
        for n in range(len(data[i])):
            x = data[i][n].loc[data[i][n]['Industry'] == tsearch]
            x = x[stat].values
            container.append(x)
    return container # Return a list of individual values

def sample_crime_cleaner(data,crime):
    container = []
    for i in data:
        for n in range(len(data[i])):
            y = data[i][n][crime]
            try:
                y = y.values[0]
            except:
                y = y.values
            container.append(y)
    return container

In [1]:
# test new search function for large sample databases
def test_search(county,state):
    db10 = db2010.loc[(db2010['State Abbreviation'] == state) & (db2010['GU Name'] == county)]
    db12 = db2012.loc[(db2012['State Abbreviation'] == state) & (db2012['GU Name'] == county)]
    db14 = db2014.loc[(db2014['State Abbreviation'] == state) & (db2014['GU Name'] == county)]
    db16 = db2016.loc[(db2016['State Abbreviation'] == state) & (db2016['GU Name'] == county)]
    db18 = economic18.loc[(economic18['State Abbreviation'] == state) & (economic18['GU Name'] == county)]
    return pd.concat([db10,db12,db14,db16])

def test_func(new_world,keys,s,n):
    bin = []
    x = new_world.loc[new_world['Industry'] == n]
    x = x[s].tolist()
    for a in x:
        bin.append(a)
    return bin
def test_funk(data,crime):
    y = [2010,2012,2014,2016]
    bin = []
    for a in keys:
        for b in y:
            x = bedrock[a].loc[(bedrock[a]['Year'] == b)][:1][crime].tolist()
            for c in x:
                bin.append(c)
    return bin
def big_industry_bot(samp_db):
        # Select most common industries from sample
    trash = []
    industry_all = []  # List of all unique industries in sample
    for a in keys:
        for b in samp_db[a]['Industry']:
            trash.append(b)
    for a in trash:
        if a not in industry_all:
            industry_all.append(a)
    base_dic = {}
    for a in industry_all:
        base_dic[a] = [trash.count(a)]

    base99 = pd.DataFrame(data=base_dic)
    base99 = base99.transpose()
    base99 = base99.nlargest(6,0)
    base99 = base99.transpose()
    industry = base99.columns.tolist()
    industry = industry[1:] # Remove 'all industries' from count
    return industry